In [ ]:
""" 
Graph: Strongly Connected Components

Find and count the strongly connected components in a given directed graph. 

The strongly connected components of a graph : 
is a partition of the vertices into subsets (maximal) 
such that each subset is strongly connected.
""" 


### 分開寫做法

In [2]:
class GraphVertex:
    def __init__(self, label):
        self.label = label       # 頂點標識符
        self.edges = []          # 鄰接頂點列表
        self.visited = False     # 訪問標記
        self.finish_time = -1    # 完成時間
        
    def __str__(self):
        return f"{self.label}: " + ", ".join(f"{self.label}->{e.label}" for e in self.edges)

def initial_graph(n: int, edge_list: list) -> list:
    """初始化有向圖"""
    graph = []
    for i in range(n):
        graph.append(GraphVertex(i))
    
    for e in edge_list:
        if 0 <= e[0] < n and 0 <= e[1] < n:
            graph[e[0]].edges.append(graph[e[1]])
    
    return graph


################################################################################
def create_transpose_graph(graph: list) -> list:
    """創建反向圖（轉置圖）"""
    n = len(graph)
    transpose = []
    
    # 創建新的頂點
    for i in range(n):
        transpose.append(GraphVertex(i))
    
    # 反向所有邊
    for v in graph:
        for neighbor in v.edges:
            transpose[neighbor.label].edges.append(transpose[v.label])
    
    return transpose

def DFS_visit(vertex: GraphVertex, time: list, finish_order: list):
    """DFS訪問單個頂點"""
    time[0] += 1
    vertex.visited = True
    
    for neighbor in vertex.edges:
        if not neighbor.visited:
            DFS_visit(neighbor, time, finish_order)
    
    # 記錄完成時間並添加到完成順序列表
    time[0] += 1
    vertex.finish_time = time[0]
    finish_order.append(vertex)

def DFS_first_pass(graph: list) -> list:
    """第一次DFS：計算完成時間"""
    time = [0]
    finish_order = []
    
    # 重置所有頂點狀態
    for v in graph:
        v.visited = False
        v.finish_time = -1
    
    for vertex in graph:
        if not vertex.visited:
            DFS_visit(vertex, time, finish_order)
    
    return finish_order

def DFS_component(vertex: GraphVertex, component: list):
    """DFS找到一個強連接組件的所有頂點"""
    vertex.visited = True
    component.append(vertex)
    
    for neighbor in vertex.edges:
        if not neighbor.visited:
            DFS_component(neighbor, component)

def DFS_second_pass(transpose_graph: list, finish_order: list) -> list:
    """第二次DFS：在轉置圖上找強連接組件"""
    # 重置轉置圖的所有頂點狀態
    for v in transpose_graph:
        v.visited = False
    
    strongly_connected_components = []
    
    # 按完成時間的逆序處理頂點
    for vertex in reversed(finish_order):
        transpose_vertex = transpose_graph[vertex.label]
        
        if not transpose_vertex.visited:
            current_component = []
            DFS_component(transpose_vertex, current_component)
            strongly_connected_components.append(current_component)
    
    return strongly_connected_components

def find_strongly_connected_components(graph: list) -> list:
    """Kosaraju算法：找到所有強連接組件"""
    # 第一步：在原圖上做DFS，記錄完成時間
    finish_order = DFS_first_pass(graph)
    
    # 第二步：創建轉置圖
    transpose = create_transpose_graph(graph)
    
    # 第三步：在轉置圖上按完成時間逆序做DFS
    components = DFS_second_pass(transpose, finish_order)
    
    return components

def count_strongly_connected_components(graph: list) -> int:
    """計算強連接組件的數量"""
    components = find_strongly_connected_components(graph)
    return len(components)

# 使用範例
if __name__ == "__main__":
    # 範例：創建圖並找強連接組件
    edge_list = [
        [0, 1], [1, 2], [2, 0],  # 強連接組件: {0, 1, 2}
        [3, 4],                   # 弱連接: 3->4
        [2, 3]                    # 連接不同組件
    ]
    
    graph = initial_graph(5, edge_list)
    components = find_strongly_connected_components(graph)
    
    print(f"找到 {len(components)} 個強連接組件:")
    for i, component in enumerate(components, 1):
        labels = sorted([v.label for v in component])
        print(f"組件 {i}: {labels}")

找到 3 個強連接組件:
組件 1: [0, 1, 2]
組件 2: [3]
組件 3: [4]


###  寫一起做法 

In [7]:
class GraphVertex:
    def __init__(self, label):
        self.label = label
        self.edges = []
        self.visited = False

def kosaraju_scc_with_vertex(edge_list: list) -> list:
    """
    使用 GraphVertex 的精簡強連接組件算法
    edge_list: 邊列表 [[u,v], ...]
    自動從邊列表計算頂點數量
    """
    # 處理空圖情況
    if not edge_list:
        return []
    
    # 自動計算頂點數量
    n = max(max(u, v) for u, v in edge_list) + 1
    # 建立原圖和反向圖
    graph = [GraphVertex(i) for i in range(n)]
    reverse_graph = [GraphVertex(i) for i in range(n)]
    
    for u, v in edge_list:
        graph[u].edges.append(graph[v])
        reverse_graph[v].edges.append(reverse_graph[u])
    
    # 第一次 DFS：記錄完成順序
    finish_stack = []
    
    def dfs1(vertex):
        vertex.visited = True
        for neighbor in vertex.edges:
            if not neighbor.visited:
                dfs1(neighbor)
        finish_stack.append(vertex)
    
    for vertex in graph:
        if not vertex.visited:
            dfs1(vertex)
    
    # 第二次 DFS：在反向圖找組件
    components = []
    
    def dfs2(vertex, component):
        vertex.visited = True
        component.append(vertex.label)
        for neighbor in vertex.edges:
            if not neighbor.visited:
                dfs2(neighbor, component)
    
    for vertex in reversed(finish_stack):
        reverse_vertex = reverse_graph[vertex.label]
        if not reverse_vertex.visited:
            component = []
            dfs2(reverse_vertex, component)
            components.append(component)
    
    return components



In [9]:
# if __name__ == "__main__":
    
#     print("=== 測試案例 1: 基本強連接組件 ===")
#     # 包含一個強連接環和單向連接
#     edge_list1 = [[0,1], [1,2], [2,0], [3,4], [2,3]]
#     components1 = kosaraju_scc_with_vertex(edge_list1)
#     print(f"邊列表: {edge_list1}")
#     print(f"強連接組件: {components1}")
#     print(f"預期: [[0,1,2], [3], [4]] - 三個組件")
#     print()
    
#     print("=== 測試案例 2: 多個強連接環 ===")
#     # 三個獨立的強連接環
#     edge_list2 = [
#         [0,1], [1,2], [2,0],  # 環1: 0->1->2->0
#         [3,4], [4,5], [5,3],  # 環2: 3->4->5->3  
#         [6,7], [7,8], [8,6],  # 環3: 6->7->8->6
#         [2,3], [5,6]          # 跨環連接
#     ]
#     components2 = kosaraju_scc_with_vertex(edge_list2)
#     print(f"邊列表: {edge_list2}")
#     print(f"強連接組件: {components2}")
#     print(f"預期: [[0,1,2], [3,4,5], [6,7,8]] - 三個環形組件")
#     print()
    
#     print("=== 測試案例 3: 純線性圖（無環） ===")
#     # 沒有環路，每個頂點自成組件
#     edge_list3 = [[0,1], [1,2], [2,3], [3,4]]
#     components3 = kosaraju_scc_with_vertex(edge_list3)
#     print(f"邊列表: {edge_list3}")
#     print(f"強連接組件: {components3}")
#     print(f"預期: [[0], [1], [2], [3], [4]] - 五個單頂點組件")
#     print()
    
#     print("=== 測試案例 4: 複雜混合圖 ===")
#     # 包含環路、分支、匯聚的複雜圖
#     edge_list4 = [
#         [0,1], [1,0],         # 小環: 0<->1
#         [1,2], [2,3], [3,2], # 分支到環: 2<->3
#         [3,4], [4,5],         # 線性延伸
#         [0,6], [6,7], [7,6]   # 分支到另一個環: 6<->7
#     ]
#     components4 = kosaraju_scc_with_vertex(edge_list4)
#     print(f"邊列表: {edge_list4}")
#     print(f"強連接組件: {components4}")
#     print(f"預期: [[0,1], [2,3], [6,7], [4], [5]] - 五個組件")
#     print()
    
#     print("=== 測試案例 5: 單個大環 ===")
#     # 所有頂點形成一個大環
#     edge_list5 = [[0,1], [1,2], [2,3], [3,4], [4,0]]
#     components5 = kosaraju_scc_with_vertex(edge_list5)
#     print(f"邊列表: {edge_list5}")
#     print(f"強連接組件: {components5}")
#     print(f"預期: [[0,1,2,3,4]] - 一個大組件")
#     print()
    
#     print("=== 測試案例 6: 空圖 ===")
#     # 沒有邊的情況
#     edge_list6 = []
#     components6 = kosaraju_scc_with_vertex(edge_list6)
#     print(f"邊列表: {edge_list6}")
#     print(f"強連接組件: {components6}")
#     print(f"預期: [] - 無組件")
#     print()
    
#     print("=== 測試案例 7: 單邊圖 ===")
#     # 只有一條邊
#     edge_list7 = [[0,1]]
#     components7 = kosaraju_scc_with_vertex(edge_list7)
#     print(f"邊列表: {edge_list7}")
#     print(f"強連接組件: {components7}")
#     print(f"預期: [[0], [1]] - 兩個單頂點組件")
#     print()
    
#     print("=== 測試案例 8: 自環圖 ===")
#     # 包含自環的情況
#     edge_list8 = [[0,0], [1,2], [2,1], [2,3]]
#     components8 = kosaraju_scc_with_vertex(edge_list8)
#     print(f"邊列表: {edge_list8}")
#     print(f"強連接組件: {components8}")
#     print(f"預期: [[0], [1,2], [3]] - 三個組件（自環算強連接）")
#     print()

### adj version : most probible 

In [10]:
class Solution:
    def kosaraju(self, adj):
        """
        找出有向圖中強連通分量的數量
        adj: 鄰接表，adj[i] 是從頂點 i 出發的所有鄰居
        返回：強連通分量的數量
        """
        n = len(adj)
        if n == 0:
            return 0
        
        # 建立反向圖
        reverse_adj = [[] for _ in range(n)]
        for u in range(n):
            for v in adj[u]:
                reverse_adj[v].append(u)
        
        # 第一次 DFS：記錄完成順序
        visited = [False] * n
        finish_stack = []
        
        def dfs1(vertex):
            visited[vertex] = True
            for neighbor in adj[vertex]:
                if not visited[neighbor]:
                    dfs1(neighbor)
            finish_stack.append(vertex)
        
        # 對所有未訪問的頂點進行第一次 DFS
        for i in range(n):
            if not visited[i]:
                dfs1(i)
        
        # 第二次 DFS：在反向圖中找強連通分量
        visited = [False] * n  # 重置訪問標記
        scc_count = 0
        
        def dfs2(vertex):
            visited[vertex] = True
            for neighbor in reverse_adj[vertex]:
                if not visited[neighbor]:
                    dfs2(neighbor)
        
        # 按照完成時間的逆序進行第二次 DFS
        while finish_stack:
            vertex = finish_stack.pop()
            if not visited[vertex]:
                dfs2(vertex)
                scc_count += 1  # 找到一個新的強連通分量
        
        return scc_count

